In [58]:
#for handling data
import numpy as np
import pandas as pd
import math

#for scraping
import requests
from bs4 import BeautifulSoup
import re
import json
import string

#for concurrency
from queue import Queue, SimpleQueue
from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor
import threading

import time
import copy

### Strategy for scraping

- All of the required data is contained in a script tag which begins with the text "window.__INITIAL_STATE__"

- First I have extracted all script tags from the soup

- Then I have extracted the required script tag (by checking if it begins with the "window.__INITIAL_STATE__" text)

- Then I perform a bit of cleaning before loading the required data as json

Note - Each of the script tags only contain 20 of the items even though we see 30 items per page. But the remaining items are available by making requests to the next page 

### Logic 
For each label keep making requests for each page till you don't get a status 200 code (this does not seem to work, the pages which do not have any data also get a success status code)



In [50]:
def extract_json(response):
    json_data=json.loads(response.content)
    return(json_data['data'])

%%time
base_url="https://www.1mg.com/drugs-all-medicines"
label=list(string.ascii_lowercase)
page=list(range(1,1000))
results=[]
for i in page:
    for j in ['a','b']:
        response=requests.get(f'{base_url}?page={str(i)}&label={str(j)}')
        soup=BeautifulSoup(response.content, "html.parser")
        if (response.status_code==200) and (extract_json(soup)['skus']):
            results.append(extract_json(BeautifulSoup(response.content, "html.parser")))
        else:
            break;

### Trying to implement the Producer/Consumer model for Threading using concurrent.futures

In [51]:
tasks=Queue()
results=SimpleQueue()

In [29]:
base_url = "https://www.1mg.com/pharmacy_api_gateway/v4/drug_skus/by_prefix"
labels = list(string.ascii_lowercase)
session = requests.Session()

In [18]:
# get number of skus for the given alphabet
def get_number_skus(alphabet):
    resp = session.get(f'{base_url}?prefix_term={alphabet}&page=1&per_page=1')
    total_count = json.loads(resp.content)['meta']['total_count']
    return total_count

In [19]:
# get number of pages to traverse for total sku count
def get_number_pages(total_count, results_per_request):
    return math.ceil(total_count/results_per_request)

In [56]:
# creating tasks
results_per_request = 50 # can be 1-50
for alpha in labels:
    pages = get_number_pages(get_number_skus(alpha), results_per_request)
    for page in range(1, pages+1):
        tasks.put(f'{base_url}?prefix_term={alpha}&page={page}&per_page={results_per_request}')
    time.sleep(5)
    

In [57]:
thread_local = threading.local()

def get_session():
    if not hasattr(thread_local, 'session'):
        thread_local.session = requests.Session()
    return thread_local.session

In [61]:
tasks_temp = copy.copy(tasks)

In [72]:
global j
j=0
def scraping_worker(tasks,results):
    
    session = get_session()
    while True:
        try:
            task = tasks.get(block=False) 
        except queue.Empty: #
            print('Queue is empty! My work here is done. Exiting.')
            return
        tasks.task_done()
        response=session.get(task)
        time.sleep(5)
        if (response.status_code==200) and (extract_json(response)['skus']):
            results.put(extract_json(response))
            j+=1
            print(f"result {j} stored")
        

In [73]:
%%time
with ThreadPoolExecutor(max_workers=15) as ex:
    futures = [
        ex.submit(scraping_worker, tasks, results) for _ in range(15)
    ]

CPU times: user 4.25 ms, sys: 2.92 ms, total: 7.16 ms
Wall time: 13 ms


In [ ]:
all([f.done() for f in futures])

True

In [74]:
tasks.qsize()

0

In [67]:
results.qsize()

139

In [ ]:
results

### Stuff to record for each card - 

1. Id and Skuid (for future fetching of data)

2. Name

3. Prescription req
    
4. is_discontinued

5. manufacturer

6. Type

7. Pack size

8. short_composition

9. rx_required.header

In [ ]:
a=results.get()
a

{'skus': [{'is_discontinued': False,
   'manufacturer_name': 'Cipla Ltd',
   'type': 'allopathy',
   'price': 271.52,
   'name': 'Alfusin Tablet PR',
   'id': 41805,
   'sku_id': 41805,
   'available': True,
   'pack_size_label': 'strip of 15 Tablet pr',
   'rx_required': {'header': 'Prescription Required',
    'icon_url': 'https://onemg.gumlet.io/image/upload/w_20,h_20/q_auto,f_auto/rx_icon.png'},
   'slug': '/drugs/alfusin-tablet-pr-41805',
   'short_composition': 'Alfuzosin (10mg)',
   'image_url': 'https://onemg.gumlet.io/image/upload/a_ignore,w_380,h_380,c_fit,q_auto,f_auto/v1537758726/bdv6ssvviagsmrj57ucq.jpg',
   'in_stock': {'add': {'action_label': 'Adding..',
     'final_label': 'ADDED',
     'label': 'ADD'},
    'cart_actions': {'dweb': ['add'], 'mweb': ['add']}},
   'quantity': 15},
  {'is_discontinued': False,
   'manufacturer_name': 'Aurz Pharmaceuticals Pvt Ltd',
   'type': 'allopathy',
   'price': 69.8,
   'name': 'Axcet M Tablet',
   'id': 234101,
   'sku_id': 234101,
 

In [ ]:
final_df=pd.DataFrame()
for i in range(0,results.qsize()):
    result=results.get()
    temp_df=pd.DataFrame([[i.get('sku_id'),
                   i.get('name'),
                   i.get('manufacturer_name'),
                   i.get('type'),
                   i.get('pack_size_label'),
                   i.get('price'),
                   i.get('rx_required'),
                   i.get('short_composition'),
                   i.get('is_discontinued')] for i in result['skus']],
                 columns=['sku_id',
                 'name','manufacturer_name',
                 'type','pack_size_label',
                 'price','rx_required',
                 'short_composition','is_discontinued'])
    final_df=pd.concat([final_df,temp_df],axis='rows',ignore_index=True)
    

In [ ]:
final_df['name'].sort_values()

3312                   A 1 5mg Tablet
5929                 A 250 Suspension
10016        A 3 100 mg/500 mg Tablet
10989           A Arti 60mg Injection
10035      A Arti L 80mg/480mg Tablet
                     ...             
46685               Hyzer 10mg Tablet
46618              Hyzine 25mg Tablet
46873                Hyzix 10mg Syrup
47172    Hyzol D 30mg/40mg Capsule SR
47210            Hyzolid 600mg Tablet
Name: name, Length: 47216, dtype: object

In [ ]:
final_df.to_csv("drugs.csv",index=False)

In [ ]:
a=requests.get("https://www.1mg.com/search/all?name=orid")

BeautifulSoup(a.content,"html.parser")

<!DOCTYPE html>

<html lang="en">
<head>
<meta charset="utf-8"/>
<meta content="initial-scale=1, maximum-scale=1, user-scalable=no, width=device-width height=device-height" name="viewport"/>
<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
<meta content="on" http-equiv="x-dns-prefetch-control"/>
<meta content="#FFF3E3" name="theme-color"/>
<!-- Windows Phone -->
<meta content="#FFF3E3" name="msapplication-navbutton-color"/>
<!-- iOS Safari -->
<meta content="#FFF3E3" name="apple-mobile-web-app-status-bar-style"/>
<meta content="yes" name="apple-mobile-web-app-capable"/>
<!-- Dynamic seo tags -->
<title>Online Pharmacy India | Buy Medicines from India's Trusted Medicine Store: 1mg.com</title>
<meta content="online pharmacy, online medicine, medicine online, online medical store, buy medicine online, online pharmacy India, buy medicine online India, online chemist, online medicine purchase" data-react-helmet="true" name="keywords"/>
<meta content="Online Pharmacy India | B